In [45]:
import pandas as pd
import numpy as np

#### train 데이터 필지 뽑아서 행정동&필지번호 합치기

In [46]:
raw_data = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/raw/필지단위 소상공인 매출등급 정보.csv', encoding='cp949')
match = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/필지번호_행정동.csv', encoding='cp949')
test = pd.read_excel('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/raw/소상공인 매출등급 예상 대상 필지.xlsx')

In [ ]:
# test 데이터는 202301과 202302가 열별로 되어있어 각각 데이터프레임 만든 후 raw데이터 밑으로 붙이기
test_2301 = test[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2302 = test[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2301['DATA_CRTR_YM'] = 202301
test_2302['DATA_CRTR_YM'] = 202302

C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_21592\2735373568.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2301['DATA_CRTR_YM'] = 202301
C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_21592\2735373568.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2302['DATA_CRTR_YM'] = 202302


In [ ]:
test_2301 = test_2301[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD']]
test_2302 = test_2302[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD']]

In [ ]:
raw_data = pd.concat([raw_data,test_2301, test_2302])

In [ ]:
print(raw_data)
print(match.head())

      DATA_CRTR_YM            LT_UNQ_NO  STDG_EMD_CD INDUSTRY_CD  SLS_GRD
0           202201  1111010100100030100     11110101         A03      4.0
1           202201  1111010100100480000     11110101         A01      2.0
2           202201  1111010100100590001     11110101         A11      3.0
3           202201  1111010100100590005     11110101         A03      2.0
4           202201  1111010100100690000     11110101         A01      2.0
...            ...                  ...          ...         ...      ...
9995        202302  1130510100100830004     11305101         A03      NaN
9996        202302  1135010300107450000     11350103         A13      NaN
9997        202302  1150010300103400017     11500103         A06      NaN
9998        202302  1144012700116070000     11440127         A03      NaN
9999        202302  1126010400100770063     11260104         A03      NaN

[1176040 rows x 5 columns]
                   PNU  SGG_CD SGG_NM    EMD_CD EMD_NM    ADM_CD ADM_NM
0  111101010

In [ ]:
# 신사동이 관악구, 강남구 둘 다에 있어서 처리해줘야함. 이외에 구는 경계에 걸친 애들
for i in range(len(match)):
    if match['ADM_NM'][i] == '신사동':
        if (match['SGG_NM'][i] == '관악구') or (match['SGG_NM'][i] == '동작구'):
            match['ADM_NM'][i] = '신사동1'
        elif (match['SGG_NM'][i] == '강남구') or (match['SGG_NM'][i] == '서초구') or (match['SGG_NM'][i] == '용산구')or (match['SGG_NM'][i] == '성동구'):
            match['ADM_NM'][i] = '신사동2'

C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_21592\568015034.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match['ADM_NM'][i] = '신사동2'
C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_21592\568015034.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match['ADM_NM'][i] = '신사동1'


##### raw데이터에서 매칭 안되는 필지번호는 분리

In [ ]:
X = pd.DataFrame(raw_data['LT_UNQ_NO'].unique())
Y = pd.DataFrame(match['PNU'].unique())
PNU_NOTIN_data = pd.DataFrame(set(raw_data['LT_UNQ_NO'].unique())-set(match['PNU'].unique()))
PNU_IN_data = pd.DataFrame(set(raw_data['LT_UNQ_NO'].unique()) & set(match['PNU'].unique()))
PNU_NOTIN_data.rename(columns={0:'PNU'}, inplace=True)
PNU_IN_data.rename(columns={0:'PNU'}, inplace=True)

In [ ]:
print(len(X))
print(len(PNU_IN_data) + len(PNU_NOTIN_data))
print(len(PNU_IN_data))
print(len(PNU_NOTIN_data))

74303
74303
74199
104


In [ ]:
raw_in_match = raw_data[raw_data['LT_UNQ_NO'].isin(PNU_IN_data['PNU'])]
raw_notin_match = raw_data[~raw_data['LT_UNQ_NO'].isin(PNU_IN_data['PNU'])]

In [ ]:
print(len(raw_data))
print(len(raw_in_match) + len(raw_notin_match))
print(len(raw_in_match))
print(len(raw_notin_match))

1176040
1176040
1172902
3138


In [ ]:
# 데이터에 있는 필지번호와 match를 merge
df = pd.merge(PNU_IN_data, match, how='left')

# 중복 필지 중 하나만 남기고 제거
df.drop_duplicates(subset='PNU', keep='first', inplace=True, ignore_index=False)

In [ ]:
print(len(PNU_IN_data))
print(len(df))

74199
74199


In [ ]:
# raw데이터에 행정동명 합치기
raw_in_match = pd.merge(raw_in_match, df[['PNU', 'ADM_CD', 'ADM_NM']], how='left', left_on = 'LT_UNQ_NO', right_on = 'PNU')

In [ ]:
raw_in_match = raw_in_match[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'ADM_CD', 'ADM_NM', 'INDUSTRY_CD', 'SLS_GRD']]

In [ ]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 1172901
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
dtypes: float64(1), int64(4), object(2)
memory usage: 71.6+ MB


#### 점포수 데이터 raw에 붙이기

In [ ]:
# raw_in_match에서 '상일1동', '상일2동'이 데이터에서는 '상일동'. 따라서 raw_in_match의 '상일1동', '상일2동'을 '상일동'으로 바꾸어줘야함
raw_in_match['ADM_NM'][raw_in_match['ADM_NM'] == '상일1동'] = '상일동'
raw_in_match['ADM_NM'][raw_in_match['ADM_NM'] == '상일2동'] = '상일동'

C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_21592\1287381924.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_in_match['ADM_NM'][raw_in_match['ADM_NM'] == '상일2동'] = '상일동'


In [ ]:
num_store = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/서울시_상권분석데이터/행정동별_점포수_1.csv', encoding = 'cp949')
num_store.head()

,행정구역,생활밀접업종,22-1전체점포수,22-1프랜차이즈점포수,22-1일반점포수,22-2전체점포수,22-2프랜차이즈점포수,22-2일반점포수,22-3전체점포수,22-3프랜차이즈점포수,22-3일반점포수,22-4전체점포수,22-4프랜차이즈점포수,22-4일반점포수,23-1전체점포수,23-1프랜차이즈점포수,23-1일반점포수
0,청운효자동,전체,296,5,291,298,5,293,305,6,299,308,6,302,312,6,306
1,사직동,전체,991,121,870,996,118,878,994,120,874,"1,001",121,880,995,117,878
2,삼청동,전체,257,28,229,259,29,230,259,30,229,266,30,236,270,29,241
3,부암동,전체,132,15,117,137,15,122,135,14,121,136,14,122,133,13,120
4,평창동,전체,144,15,129,144,15,129,139,15,124,141,15,126,143,16,127


In [ ]:
# 업종 코드와 업종명을 매핑하는 딕셔너리 생성
industry_mapping = {
    'A01': '한식음식점',
    'A02': '치킨전문점',
    'A03': '전체',
    'A10': '전체',
    'A04': '제과점',
    'A05': '패스트푸드점',
    'A07': '패스트푸드점',
    'A06': '분식전문점',
    'A08': '커피-음료',
    'A09': '커피-음료',
    'A15': '커피-음료',
    'A11': '중식음식점',
    'A12': '호프-간이주점',
    'A13': '일식음식점',
    'A14': '양식음식점'
}

# '생활밀접업종' 열 추가 및 매핑
raw_in_match['생활밀접업종'] = raw_in_match['INDUSTRY_CD'].map(industry_mapping)

print(raw_in_match)


         DATA_CRTR_YM            LT_UNQ_NO  STDG_EMD_CD    ADM_CD ADM_NM  \
0              202201  1111010100100030100     11110101  11010720  청운효자동   
1              202201  1111010100100480000     11110101  11010720  청운효자동   
2              202201  1111010100100590001     11110101  11010720  청운효자동   
3              202201  1111010100100590005     11110101  11010720  청운효자동   
4              202201  1111010100100690000     11110101  11010720  청운효자동   
...               ...                  ...          ...       ...    ...   
1172897        202302  1130510100100830004     11305101  11090710    송중동   
1172898        202302  1135010300107450000     11350103  11110560   공릉2동   
1172899        202302  1150010300103400017     11500103  11160630   화곡8동   
1172900        202302  1144012700116070000     11440127  11140740    상암동   
1172901        202302  1126010400100770063     11260104  11070640    묵1동   

        INDUSTRY_CD  SLS_GRD 생활밀접업종  
0               A03      4.0     전체  
1          

In [ ]:
# 분기별로 merge
raw_22_1 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] <= 202203], num_store[['행정구역', '생활밀접업종', '22-1전체점포수', '22-1프랜차이즈점포수', '22-1일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_2 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202206) & (raw_in_match['DATA_CRTR_YM'] > 202203)], num_store[['행정구역', '생활밀접업종', '22-2전체점포수', '22-2프랜차이즈점포수', '22-2일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_3 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202209) & (raw_in_match['DATA_CRTR_YM'] > 202206)], num_store[['행정구역', '생활밀접업종', '22-3전체점포수', '22-3프랜차이즈점포수', '22-3일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_4 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202212) & (raw_in_match['DATA_CRTR_YM'] > 202209)], num_store[['행정구역', '생활밀접업종', '22-4전체점포수', '22-4프랜차이즈점포수', '22-4일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_23_1 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202302) & (raw_in_match['DATA_CRTR_YM'] > 202212)], num_store[['행정구역', '생활밀접업종', '23-1전체점포수', '23-1프랜차이즈점포수', '23-1일반점포수']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')

In [ ]:
# 열 이름 통일
## 22-1
col_lst = list(raw_22_1.columns)
col_lst[9] = '전체점포수'
col_lst[10] = '프랜차이즈점포수'
col_lst[11] = '일반점포수'
raw_22_1.columns = col_lst
## 22-2
col_lst = list(raw_22_2.columns)
col_lst[9] = '전체점포수'
col_lst[10] = '프랜차이즈점포수'
col_lst[11] = '일반점포수'
raw_22_2.columns = col_lst
## 22-3
col_lst = list(raw_22_3.columns)
col_lst[9] = '전체점포수'
col_lst[10] = '프랜차이즈점포수'
col_lst[11] = '일반점포수'
raw_22_3.columns = col_lst
## 22-4
col_lst = list(raw_22_4.columns)
col_lst[9] = '전체점포수'
col_lst[10] = '프랜차이즈점포수'
col_lst[11] = '일반점포수'
raw_22_4.columns = col_lst
## 23-1
col_lst = list(raw_23_1.columns)
col_lst[9] = '전체점포수'
col_lst[10] = '프랜차이즈점포수'
col_lst[11] = '일반점포수'
raw_23_1.columns = col_lst

In [ ]:
# 분기별 데이터 하나로 합치기
raw_in_match = pd.concat([raw_22_1, raw_22_2, raw_22_3, raw_22_4, raw_23_1])
raw_in_match.drop(columns='행정구역', axis=1, inplace=True)

In [ ]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 19969
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   생활밀접업종        1172902 non-null  object 
 8   전체점포수         1172902 non-null  object 
 9   프랜차이즈점포수      1172902 non-null  int64  
 10  일반점포수         1172902 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 107.4+ MB


##### A03, A10 비율 곱하기

In [ ]:
# '전체점포수'와 '일반점포수' 열에서 쉼표(,) 제거하고 문자열을 숫자로 변환
raw_in_match['전체점포수'] = raw_in_match['전체점포수'].str.replace(',', '').astype(int)
raw_in_match['일반점포수'] = raw_in_match['일반점포수'].str.replace(',', '').astype(int)

# 비율을 적용할 조건을 설정합니다.
condition_A03 = (raw_in_match['INDUSTRY_CD'] == 'A03')
condition_A10 = (raw_in_match['INDUSTRY_CD'] == 'A10')

# 데이터프레임에 조건을 적용하여 비율을 계산합니다.
raw_in_match.loc[condition_A03, ['전체점포수', '프랜차이즈점포수', '일반점포수']] *= 0.3
raw_in_match.loc[condition_A10, ['전체점포수', '프랜차이즈점포수', '일반점포수']] *= 0.036


In [ ]:
raw_in_match.isna().sum()

DATA_CRTR_YM        0
LT_UNQ_NO           0
STDG_EMD_CD         0
ADM_CD              0
ADM_NM              0
INDUSTRY_CD         0
SLS_GRD         19970
생활밀접업종              0
전체점포수               0
프랜차이즈점포수            0
일반점포수               0
dtype: int64

#### 인구수 데이터 합치기

In [ ]:
num_pop = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/서울시_상권분석데이터/행정동별_인구수_1.csv', encoding='cp949')
num_pop.head()

,행정구역,22-1길단위유동인구,22-1주거인구,22-1직장인구,22-2길단위유동인구,22-2주거인구,22-2직장인구,22-3길단위유동인구,22-3주거인구,22-3직장인구,22-4길단위유동인구,22-4주거인구,22-4직장인구,23-1길단위유동인구,23-1주거인구,23-1직장인구
0,청운효자동,"14,929",51,44,"16,209",51,44,"14,843",51,44,"15,269",51,44,"14,633",54,44
1,사직동,"29,283",82,412,"34,630",82,412,"33,738",82,412,"34,815",82,412,"33,445",73,412
2,삼청동,"5,905",19,42,"6,885",19,42,"6,302",19,42,"6,649",19,42,"6,696",19,42
3,부암동,"5,778",45,19,"6,011",45,19,"5,468",45,19,"5,248",45,19,"4,970",44,19
4,평창동,987,22,28,986,22,28,975,22,28,"1,019",22,28,989,21,28


In [ ]:
# 분기별로 merge
raw_22_1 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] <= 202203], num_pop[['행정구역', '22-1길단위유동인구', '22-1주거인구', '22-1직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_2 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202206) & (raw_in_match['DATA_CRTR_YM'] > 202203)], num_pop[['행정구역', '22-2길단위유동인구', '22-2주거인구', '22-2직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_3 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202209) & (raw_in_match['DATA_CRTR_YM'] > 202206)], num_pop[['행정구역', '22-3길단위유동인구', '22-3주거인구', '22-3직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_4 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202212) & (raw_in_match['DATA_CRTR_YM'] > 202209)], num_pop[['행정구역', '22-4길단위유동인구', '22-4주거인구', '22-4직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_23_1 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202302) & (raw_in_match['DATA_CRTR_YM'] > 202212)], num_pop[['행정구역', '23-1길단위유동인구', '23-1주거인구', '23-1직장인구']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')

In [ ]:
# 열 이름 통일
## 22-1
col_lst = list(raw_22_1.columns)
col_lst[12] = '길단위유동인구'
col_lst[13] = '주거인구'
col_lst[14] = '직장인구'
raw_22_1.columns = col_lst
## 22-2
col_lst = list(raw_22_2.columns)
col_lst[12] = '길단위유동인구'
col_lst[13] = '주거인구'
col_lst[14] = '직장인구'
raw_22_2.columns = col_lst
## 22-3
col_lst = list(raw_22_3.columns)
col_lst[12] = '길단위유동인구'
col_lst[13] = '주거인구'
col_lst[14] = '직장인구'
raw_22_3.columns = col_lst
## 22-4
col_lst = list(raw_22_4.columns)
col_lst[12] = '길단위유동인구'
col_lst[13] = '주거인구'
col_lst[14] = '직장인구'
raw_22_4.columns = col_lst
## 23-1
col_lst = list(raw_23_1.columns)
col_lst[12] = '길단위유동인구'
col_lst[13] = '주거인구'
col_lst[14] = '직장인구'
raw_23_1.columns = col_lst

In [ ]:
# 분기별 데이터 하나로 합치기
raw_in_match = pd.concat([raw_22_1, raw_22_2, raw_22_3, raw_22_4, raw_23_1])
raw_in_match.drop(columns='행정구역', axis=1, inplace=True)

In [ ]:
# '길단위유동인구'와 '직장인구' 열에서 쉼표(,) 제거하고 문자열을 숫자로 변환
raw_in_match['길단위유동인구'] = raw_in_match['길단위유동인구'].str.replace(',', '').astype(int)
raw_in_match['직장인구'] = raw_in_match['직장인구'].str.replace(',', '').astype(int)

In [ ]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 19969
Data columns (total 14 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   생활밀접업종        1172902 non-null  object 
 8   전체점포수         1172902 non-null  float64
 9   프랜차이즈점포수      1172902 non-null  float64
 10  일반점포수         1172902 non-null  float64
 11  길단위유동인구       1172902 non-null  int32  
 12  주거인구          1172902 non-null  int64  
 13  직장인구          1172902 non-null  int32  
dtypes: float64(4), int32(2), int64(5), object(3)
memory usage: 125.3+ MB


#### 개폐업수 합치기

In [ ]:
open_close = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/서울시_상권분석데이터/행정동별_개폐업수_1.csv', encoding='cp949')
open_close.head()

,행정구역,생활밀접업종,22-1개업수,22-1폐업수,22-1개업률,22-1폐업률,22-2개업수,22-2폐업수,22-2개업률,22-2폐업률,...,22-3개업률,22-3폐업률,22-4개업수,22-4폐업수,22-4개업률,22-4폐업률,23-1개업수,23-1폐업수,23-1개업률,23-1폐업률
0,청운효자동,제과점,3,0,13.0,0.0,0,0,0.0,0.0,...,0.0,9.5,0,2,0.0,10.5,1,0,5.0,0.0
1,사직동,제과점,1,0,3.1,0.0,0,0,0.0,0.0,...,3.1,3.1,0,0,0.0,0.0,0,0,0.0,0.0
2,삼청동,제과점,0,0,0.0,0.0,0,1,0.0,7.1,...,7.7,15.4,2,0,13.3,0.0,0,0,0.0,0.0
3,부암동,제과점,1,0,14.3,0.0,0,0,0.0,0.0,...,12.5,0.0,0,0,0.0,0.0,0,1,0.0,14.3
4,평창동,제과점,0,0,0.0,0.0,0,0,0.0,0.0,...,0.0,0.0,0,1,0.0,20.0,0,0,0.0,0.0


In [ ]:
# 분기별로 merge
raw_22_1 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] <= 202203], open_close[['행정구역', '생활밀접업종', '22-1개업수', '22-1폐업수', '22-1개업률', '22-1폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_2 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202206) & (raw_in_match['DATA_CRTR_YM'] > 202203)], open_close[['행정구역', '생활밀접업종', '22-2개업수', '22-2폐업수', '22-2개업률', '22-2폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_3 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202209) & (raw_in_match['DATA_CRTR_YM'] > 202206)], open_close[['행정구역', '생활밀접업종', '22-3개업수', '22-3폐업수', '22-3개업률', '22-3폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_22_4 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202212) & (raw_in_match['DATA_CRTR_YM'] > 202209)], open_close[['행정구역', '생활밀접업종', '22-4개업수', '22-4폐업수', '22-4개업률', '22-4폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')
raw_23_1 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202302) & (raw_in_match['DATA_CRTR_YM'] > 202212)], open_close[['행정구역', '생활밀접업종', '23-1개업수', '23-1폐업수', '23-1개업률', '23-1폐업률']], left_on=['ADM_NM', '생활밀접업종'], right_on = ['행정구역', '생활밀접업종'], how='left')

In [ ]:
# 열 이름 통일
## 22-1
col_lst = list(raw_22_1.columns)
col_lst[15] = '개업수'
col_lst[16] = '폐업수'
col_lst[17] = '개업률'
col_lst[18] = '폐업률'
raw_22_1.columns = col_lst
## 22-2
col_lst = list(raw_22_2.columns)
col_lst[15] = '개업수'
col_lst[16] = '폐업수'
col_lst[17] = '개업률'
col_lst[18] = '폐업률'
raw_22_2.columns = col_lst
## 22-3
col_lst = list(raw_22_3.columns)
col_lst[15] = '개업수'
col_lst[16] = '폐업수'
col_lst[17] = '개업률'
col_lst[18] = '폐업률'
raw_22_3.columns = col_lst
## 22-4
col_lst = list(raw_22_4.columns)
col_lst[15] = '개업수'
col_lst[16] = '폐업수'
col_lst[17] = '개업률'
col_lst[18] = '폐업률'
raw_22_4.columns = col_lst
## 23-1
col_lst = list(raw_23_1.columns)
col_lst[15] = '개업수'
col_lst[16] = '폐업수'
col_lst[17] = '개업률'
col_lst[18] = '폐업률'
raw_23_1.columns = col_lst

In [ ]:
# 분기별 데이터 하나로 합치기
raw_in_match = pd.concat([raw_22_1, raw_22_2, raw_22_3, raw_22_4, raw_23_1])
raw_in_match.drop(columns='행정구역', axis=1, inplace=True)

In [ ]:
# 비율을 적용할 조건을 설정합니다.
condition_A03 = (raw_in_match['INDUSTRY_CD'] == 'A03')
condition_A10 = (raw_in_match['INDUSTRY_CD'] == 'A10')

# 데이터프레임에 조건을 적용하여 비율을 계산합니다.
raw_in_match.loc[condition_A03, ['개업수', '폐업수', '개업률', '폐업률']] *= 0.3
raw_in_match.loc[condition_A10, ['개업수', '폐업수', '개업률', '폐업률']] *= 0.036

In [ ]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 19969
Data columns (total 18 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   생활밀접업종        1172902 non-null  object 
 8   전체점포수         1172902 non-null  float64
 9   프랜차이즈점포수      1172902 non-null  float64
 10  일반점포수         1172902 non-null  float64
 11  길단위유동인구       1172902 non-null  int32  
 12  주거인구          1172902 non-null  int64  
 13  직장인구          1172902 non-null  int32  
 14  개업수           1172902 non-null  float64
 15  폐업수           1172902 non-null  float64
 16  개업률           1172902 non-null  float64
 17  폐업률           1172902 non-nul

#### 임대료 합치기

In [ ]:
fee_rent = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/2023 빅콘테스트/data/서울시_상권분석데이터/행정동별_임대료_1.csv', encoding='cp949')
fee_rent.head()

,행정구역,22-1 전체,22-1 1층,22-1 1층 외,22-2 전체,22-2 1층,22-2 1층 외,22-3 전체,22-3 1층,22-3 1층 외,22-4 전체,22-4 1층,22-4 1층 외,23-1 전체,23-1 1층,23-1 1층 외
0,청운효자동,"129,755","181,103","78,407","147,878","212,258","83,498","142,725","195,183","90,268","155,797","224,849","86,744","151,256","225,252","77,260"
1,사직동,"145,664","185,390","105,938","162,108","217,048","107,167","166,024","228,857","103,192","173,553","246,659","100,447","150,227","209,365","91,088"
2,삼청동,"175,042","185,890","164,194","161,693","180,283","143,103","157,572","194,483","120,661","168,139","200,614","135,664","190,228","226,225","154,231"
3,부암동,"88,340","94,506","82,173","74,155","93,432","54,879","93,334","118,701","67,967","99,441","122,603","76,278","107,082","107,082",NaN
4,평창동,"92,288","115,285","69,292","110,528","148,983","72,074","111,164","123,164","99,164","121,969","132,186","111,752","116,535",NaN,"116,535"


In [ ]:
# 분기별로 merge
raw_22_1 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] <= 202203], fee_rent[['행정구역', '22-1 전체', '22-1 1층', '22-1 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_2 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202206) & (raw_in_match['DATA_CRTR_YM'] > 202203)], fee_rent[['행정구역', '22-2 전체', '22-2 1층', '22-2 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_3 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202209) & (raw_in_match['DATA_CRTR_YM'] > 202206)], fee_rent[['행정구역', '22-3 전체', '22-3 1층', '22-3 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_22_4 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202212) & (raw_in_match['DATA_CRTR_YM'] > 202209)], fee_rent[['행정구역', '22-4 전체', '22-4 1층', '22-4 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')
raw_23_1 = pd.merge(raw_in_match[(raw_in_match['DATA_CRTR_YM'] <= 202302) & (raw_in_match['DATA_CRTR_YM'] > 202212)], fee_rent[['행정구역', '23-1 전체', '23-1 1층', '23-1 1층 외']], left_on=['ADM_NM'], right_on = ['행정구역'], how='left')

In [ ]:
# 열 이름 통일
## 22-1
col_lst = list(raw_22_1.columns)
col_lst[19] = '전체 임대료'
col_lst[20] = '1층 임대료'
col_lst[21] = '1층 외 임대료'
raw_22_1.columns = col_lst
## 22-2
col_lst = list(raw_22_2.columns)
col_lst[19] = '전체 임대료'
col_lst[20] = '1층 임대료'
col_lst[21] = '1층 외 임대료'
raw_22_2.columns = col_lst
## 22-3
col_lst = list(raw_22_3.columns)
col_lst[19] = '전체 임대료'
col_lst[20] = '1층 임대료'
col_lst[21] = '1층 외 임대료'
raw_22_3.columns = col_lst
## 22-4
col_lst = list(raw_22_4.columns)
col_lst[19] = '전체 임대료'
col_lst[20] = '1층 임대료'
col_lst[21] = '1층 외 임대료'
raw_22_4.columns = col_lst
## 23-1
col_lst = list(raw_23_1.columns)
col_lst[19] = '전체 임대료'
col_lst[20] = '1층 임대료'
col_lst[21] = '1층 외 임대료'
raw_23_1.columns = col_lst

In [ ]:
# 분기별 데이터 하나로 합치기
raw_in_match = pd.concat([raw_22_1, raw_22_2, raw_22_3, raw_22_4, raw_23_1])
raw_in_match.drop(columns='행정구역', axis=1, inplace=True)

In [ ]:
# '전체 임대료'와 '1층 임대료', '1층 외 임대료' 열에서 쉼표(,) 제거하고 문자열을 숫자로 변환
# null값 처리 후 해야함
""" 
raw_in_match['전체 임대료'] = raw_in_match['전체 임대료'].str.replace(',', '').astype(int)
raw_in_match['1층 임대료'] = raw_in_match['1층 임대료'].str.replace(',', '').astype(int)
raw_in_match['1층 외 임대료'] = raw_in_match['1층 외 임대료'].str.replace(',', '').astype(int)
"""

" \nraw_in_match['전체 임대료'] = raw_in_match['전체 임대료'].str.replace(',', '').astype(int)\nraw_in_match['1층 임대료'] = raw_in_match['1층 임대료'].str.replace(',', '').astype(int)\nraw_in_match['1층 외 임대료'] = raw_in_match['1층 외 임대료'].str.replace(',', '').astype(int)\n"

#### 점포수, 인구수, 개폐업수, 임대료 변수 다 합친 데이터 보기

In [ ]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 19969
Data columns (total 21 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   생활밀접업종        1172902 non-null  object 
 8   전체점포수         1172902 non-null  float64
 9   프랜차이즈점포수      1172902 non-null  float64
 10  일반점포수         1172902 non-null  float64
 11  길단위유동인구       1172902 non-null  int32  
 12  주거인구          1172902 non-null  int64  
 13  직장인구          1172902 non-null  int32  
 14  개업수           1172902 non-null  float64
 15  폐업수           1172902 non-null  float64
 16  개업률           1172902 non-null  float64
 17  폐업률           1172902 non-nul

In [ ]:
raw_in_match.isna().sum()

DATA_CRTR_YM        0
LT_UNQ_NO           0
STDG_EMD_CD         0
ADM_CD              0
ADM_NM              0
INDUSTRY_CD         0
SLS_GRD         19970
생활밀접업종              0
전체점포수               0
프랜차이즈점포수            0
일반점포수               0
길단위유동인구             0
주거인구                0
직장인구                0
개업수                 0
폐업수                 0
개업률                 0
폐업률                 0
전체 임대료          16703
1층 임대료          18149
1층 외 임대료        20188
dtype: int64